In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  

start_date = '2020-01-01'
end_date = '2024-11-29'

adj_close = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

print(adj_close.tail(10))

adj_close.to_csv('adj_close_prices.csv')#save file for section use offline 



[*********************100%***********************]  5 of 5 completed

Ticker            AAPL        AMZN       GOOGL        META        MSFT
Date                                                                  
2024-11-14  228.220001  211.479996  175.580002  577.159973  426.037231
2024-11-15  225.000000  202.610001  172.490005  554.080017  414.170990
2024-11-18  228.020004  201.699997  175.300003  554.400024  414.929474
2024-11-19  228.279999  204.610001  178.119995  561.090027  416.955414
2024-11-20  229.000000  202.880005  175.979996  565.520020  414.659973
2024-11-21  228.520004  198.380005  167.630005  563.090027  412.869995
2024-11-22  229.869995  197.119995  164.759995  559.140015  417.000000
2024-11-25  232.869995  201.449997  167.649994  565.109985  418.790009
2024-11-26  235.059998  207.860001  169.119995  573.539978  427.989990
2024-11-27  234.929993  205.740005  169.229996  569.200012  422.989990


In [3]:
returns = adj_close.pct_change().dropna()

returns

Ticker,AAPL,AMZN,GOOGL,META,MSFT
Date,,,,,
2020-01-03,-0.009722,-0.012139,-0.005231,-0.005291,-0.012452
2020-01-06,0.007968,0.014886,0.026654,0.018834,0.002585
2020-01-07,-0.004703,0.002092,-0.001931,0.002164,-0.009118
2020-01-08,0.016086,-0.007809,0.007118,0.010138,0.015928
2020-01-09,0.021241,0.004799,0.010498,0.014311,0.012493
...,...,...,...,...,...
2024-11-21,-0.002096,-0.022181,-0.047449,-0.004297,-0.004317
2024-11-22,0.005908,-0.006351,-0.017121,-0.007015,0.010003
2024-11-25,0.013051,0.021966,0.017541,0.010677,0.004293


In [4]:
expected_returns = returns.mean()
cov_matrix = returns.cov()

print(expected_returns, cov_matrix)

Ticker
AAPL     0.001151
AMZN     0.000885
GOOGL    0.000944
META     0.001220
MSFT     0.001007
dtype: float64 Ticker      AAPL      AMZN     GOOGL      META      MSFT
Ticker                                                  
AAPL    0.000404  0.000271  0.000268  0.000322  0.000291
AMZN    0.000271  0.000517  0.000301  0.000394  0.000298
GOOGL   0.000268  0.000301  0.000418  0.000374  0.000296
META    0.000322  0.000394  0.000374  0.000809  0.000341
MSFT    0.000291  0.000298  0.000296  0.000341  0.000373


In [5]:
lambda_risk = .5

In [6]:
n_assets = len(tickers)
weights = cp.Variable(n_assets)

In [7]:
portfolio_return = expected_returns.values @ weights
portfolio_variance = cp.quad_form(weights, cov_matrix)
objective = cp.Minimize(-portfolio_return + lambda_risk * portfolio_variance)

In [8]:
constraints = [cp.sum(weights) == 1, weights >= 0]


In [9]:
problem = cp.Problem(objective, constraints)
problem.solve()

-0.000968845028285414

In [10]:
optimal_weights = weights.value

In [11]:
print("Optimal Weights:")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {optimal_weights[i]:.4f}")
optimal_weights_df = pd.DataFrame(optimal_weights, index=tickers, columns=['Weight'])
optimal_weights_df.to_csv('optimal_weights.csv')


Optimal Weights:
AAPL: 0.7339
MSFT: -0.0000
GOOGL: -0.0000
AMZN: 0.2661
META: -0.0000


# add quadrtic function for non convex


In [12]:
non_linear_constraint_exp = cp.exp(weights[2]) <= 1.5
constraints.append(non_linear_constraint_exp)

In [13]:
problem = cp.Problem(objective, constraints)
problem.solve()


print("Optimal Weights:")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {optimal_weights[i]:.4f}")
optimal_weights_df = pd.DataFrame(optimal_weights, index=tickers, columns=['Weight'])
optimal_weights_df.to_csv('optimal_weights.csv')


Optimal Weights:
AAPL: 0.7339
MSFT: -0.0000
GOOGL: -0.0000
AMZN: 0.2661
META: -0.0000


In [14]:
import plotly.express as px

# Load the adjusted close prices from the CSV file
adj_close = pd.read_csv('adj_close_prices.csv', index_col=0, parse_dates=True)

# Create a line plot for each stock
fig = px.line(adj_close, x=adj_close.index, y=adj_close.columns, 
              labels={'value': 'Adjusted Close Price', 'variable': 'Stock'},
              title='Stock Price Trends')

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    legend_title='Stock',
    hovermode='x unified'
)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objects as go

adj_close = pd.read_csv('adj_close_prices.csv', index_col=0, parse_dates=True)

adj_close['AAPL_MA50'] = adj_close['AAPL'].rolling(window=50).mean()
adj_close['AAPL_MA200'] = adj_close['AAPL'].rolling(window=200).mean()

fig = go.Figure(data=[go.Candlestick(x=adj_close.index,
                                     open=adj_close['AAPL'].shift(1),
                                     high=adj_close['AAPL'].rolling(window=2).max(),
                                     low=adj_close['AAPL'].rolling(window=2).min(),
                                     close=adj_close['AAPL'],
                                     name='AAPL')])

fig.add_trace(go.Scatter(x=adj_close.index, y=adj_close['AAPL_MA50'], 
                         mode='lines', name='50-day MA'))
fig.add_trace(go.Scatter(x=adj_close.index, y=adj_close['AAPL_MA200'], 
                         mode='lines', name='200-day MA'))

fig.update_layout(
    title='AAPL Candlestick Chart with Moving Averages',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    legend_title='Indicator',
    hovermode='x unified'
)

fig.show()

# Return plot

In [ ]:
import plotly.express as px

returns = adj_close.pct_change().dropna()

fig_hist = px.histogram(returns, x=returns.columns, 
                        nbins=50, 
                        labels={'value': 'Daily Return', 'variable': 'Stock'},
                        title='Distribution of Daily Returns')

fig_hist.update_layout(
    xaxis_title='Daily Return',
    yaxis_title='Frequency',
    legend_title='Stock',
    hovermode='x unified'
)

fig_hist.show()

# fig_box = px.box(returns, y=returns.columns, 
#                  labels={'value': 'Daily Return', 'variable': 'Stock'},
#                  title='Box Plot of Daily Returns')

# fig_box.update_layout(
#     yaxis_title='Daily Return',
#     legend_title='Stock',
#     hovermode='x unified'
# )

# fig_box.show()

In [ ]:
import plotly.express as px
correlation_matrix = adj_close.corr()

fig_corr = px.imshow(correlation_matrix, 
                     labels=dict(color="Correlation"),
                     x=correlation_matrix.columns,
                     y=correlation_matrix.columns,
                     title='Correlation Matrix of Adjusted Close Prices')

fig_corr.update_layout(
    xaxis_title='Stock',
    yaxis_title='Stock',
    coloraxis_colorbar=dict(title="Correlation"),
    hovermode='closest'
)

fig_corr.show()